In [806]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Import Data

In [809]:
df = pd.read_csv("combined_df.csv")
df = df[
    (df['Year'] >= 2020) &
    ((df['Headliner'].str.contains('"', na=False))) & 
    (df['Genre'] != 'Family Entertainment') &
    (df['Ticket Price Min USD'] > 0) &
    (df['Ticket Price Min USD'] < df['Ticket Price Max USD'])
]
# df = df.dropna()
# # Reset index after dropping rows
# df = df.reset_index(drop=True)
df

/var/folders/5p/g82q05js0bs2xkv11dw5ksy80000gn/T/ipykernel_59103/2258568049.py:1: DtypeWarning: Columns (2,3,6,7,8,9,10,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("combined_df.csv")


,Event Date,Headliner,sp artist_name,sp artist_genre,sp followers,sp popularity,yt name,yt Channel ID,yt Title,yt Description,...,Genre,Avg. Tickets Sold,Avg. Gross USD,Avg. Event Capacity,Avg. Capacity Sold,Ticket Price Min USD,Ticket Price Max USD,Ticket Price Avg. USD,Month,day_of_week
7,2024-09-11,"Greg ""G"" Williams",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Comedy,40.0,1240.00,220.0,18%,25.0,35.0,31.00,9,2
56,2024-09-05,"""K-Pop Club Night""","""K-Pop Club Night""","['k-pop', 'k-pop girl group']",9156939.0,82.0,"""K-Pop Club Night""",UCSSEYmHc2HGnFWwHhRXY5kA,P Su,NaN,...,Pop / Rock,361.0,9113.00,1600.0,22%,22.0,35.0,25.24,9,3
60,2024-09-05,"""RuPaul's Drag Race""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Theatrical,1524.0,108716.75,3048.0,50%,25.0,99.5,71.34,9,3
80,2024-09-03,"""Avatar: The Last Airbender In Concert""","""Avatar: The Last Airbender In Concert""","['pixel', 'video game music']",1025270.0,73.0,"""Avatar: The Last Airbender In Concert""",UCugEMaKUpe62yldZ2DECTEQ,Robert Van Der Vliet,NaN,...,Pop / Rock,689.0,36846.00,1881.0,36%,20.0,125.0,53.48,9,1
93,2024-09-01,"""RuPaul's Drag Race""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Theatrical,2108.0,162809.00,2704.0,77%,49.5,99.5,77.23,9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109659,2020-01-02,"Cirque du Soleil - ""Axel""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Theatrical,3280.0,200146.17,20000.0,16%,25.0,110.0,61.01,1,3
109663,2020-01-02,"Cirque du Soleil - ""Ovo""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Theatrical,1142.0,63001.50,3908.0,29%,39.0,125.0,55.16,1,3
109672,2020-01-01,"""Voices On The Hudson"", Joseph Arthur",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Americana, Multi-Genre",89.0,2810.00,100.0,89%,30.0,35.0,31.40,1,2
109682,2020-01-01,"""Hangover Ball""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Pop / Rock,1119.0,26255.00,1700.0,65%,22.0,42.0,23.46,1,2


In [810]:
# Calculate # and % of missing values in each column
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percent})

# sort missing values
missing_df = missing_df.sort_values(by='Missing Values', ascending=False)
missing_df

,Missing Values,Percentage
yt Description,8938,99.068943
yt View Count,8870,98.315229
yt Video Count,8870,98.315229
yt name,8870,98.315229
yt Channel ID,8870,98.315229
yt Title,8870,98.315229
yt Subscriber Count,8870,98.315229
yt Published At,8870,98.315229
35 to 44 years population,8866,98.270893
45 to 54 years population,8866,98.270893


In [811]:
# just keep columns with < 80% missing values
columns_to_keep = []
for col in df.columns:
    if missing_percent[col] < 80:
        columns_to_keep.append(col)

df_filtered = df[columns_to_keep]
df_filtered.head()

,Event Date,Headliner,Year,headliner_monthly_listeners,Support_Total_Monthly_Listeners,monthly_listeners,Number of Shows,Support,Venue,City,...,Genre,Avg. Tickets Sold,Avg. Gross USD,Avg. Event Capacity,Avg. Capacity Sold,Ticket Price Min USD,Ticket Price Max USD,Ticket Price Avg. USD,Month,day_of_week
7,2024-09-11,"Greg ""G"" Williams",2024.0,0.0,0.0,NaN,1,NaN,City Winery Pittsburgh,Pittsburgh,...,Comedy,40.0,1240.00,220.0,18%,25.0,35.0,31.00,9,2
56,2024-09-05,"""K-Pop Club Night""",2024.0,0.0,0.0,NaN,1,NaN,Big Night Live,Boston,...,Pop / Rock,361.0,9113.00,1600.0,22%,22.0,35.0,25.24,9,3
60,2024-09-05,"""RuPaul's Drag Race""",2024.0,0.0,0.0,NaN,1,NaN,Altria Theater,Richmond,...,Theatrical,1524.0,108716.75,3048.0,50%,25.0,99.5,71.34,9,3
80,2024-09-03,"""Avatar: The Last Airbender In Concert""",2024.0,0.0,0.0,NaN,1,NaN,Washington Pavilion of Arts and Science - Mary...,Sioux Falls,...,Pop / Rock,689.0,36846.00,1881.0,36%,20.0,125.0,53.48,9,1
93,2024-09-01,"""RuPaul's Drag Race""",2024.0,0.0,0.0,NaN,1,NaN,DPAC,Durham,...,Theatrical,2108.0,162809.00,2704.0,77%,49.5,99.5,77.23,9,6


In [812]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9022 entries, 7 to 109685
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Event Date                       9022 non-null   object 
 1   Headliner                        9022 non-null   object 
 2   Year                             9022 non-null   float64
 3   headliner_monthly_listeners      9022 non-null   float64
 4   Support_Total_Monthly_Listeners  9022 non-null   float64
 5   monthly_listeners                1888 non-null   float64
 6   Number of Shows                  9022 non-null   int64  
 7   Support                          2029 non-null   object 
 8   Venue                            9022 non-null   object 
 9   City                             9022 non-null   object 
 10  State                            9022 non-null   object 
 11  Country                          9022 non-null   object 
 12  Market                 

In [813]:
# let consider monthly_listeners's effect, which has 79% missing value
print(df_filtered.groupby(df['monthly_listeners'].isnull())['Avg. Gross USD'].mean())

monthly_listeners
False    419099.173554
True      84256.754404
Name: Avg. Gross USD, dtype: float64


In [814]:
# corr for numerical feature
correlation_matrix = df_filtered.select_dtypes(include=['number']).corr()
print(correlation_matrix['Avg. Gross USD'].sort_values(ascending=False))

Avg. Gross USD                     1.000000
Avg. Tickets Sold                  0.872640
Avg. Event Capacity                0.786855
Ticket Price Max USD               0.568861
Support_Total_Monthly_Listeners    0.488328
monthly_listeners                  0.475429
Ticket Price Avg. USD              0.447727
Ticket Price Min USD               0.404142
Month                              0.041584
day_of_week                        0.025197
Number of Shows                    0.007792
headliner_monthly_listeners       -0.000263
Year                              -0.034602
Name: Avg. Gross USD, dtype: float64


In [815]:
# monthly_listeners takes a quite big role, 
# so we can fill na with median (with same artist) first
df_filtered = df_filtered.copy()
df_filtered.loc[:, 'monthly_listeners'] = df_filtered.groupby('Headliner')['monthly_listeners'].transform(lambda x: x.fillna(x.median()))

# then fill the remaining NaNs with the global median
df_filtered.loc[:, 'monthly_listeners'] = df_filtered['monthly_listeners'].fillna(df_filtered['monthly_listeners'].median())

# check monthly_listener missing value again
print("Missing values in monthly_listeners after final fill:", df_filtered['monthly_listeners'].isnull().sum())

Missing values in monthly_listeners after final fill: 0


In [816]:
# fill other features missing value
df_filtered = df_filtered.copy()

df_filtered.loc[:, 'Support'] = df_filtered['Support'].fillna(df_filtered['Support'].mode()[0])
df_filtered.loc[:, 'Market'] = df_filtered['Market'].fillna(df_filtered['Market'].mode()[0])
df_filtered.loc[:, 'Promoter'] = df_filtered['Promoter'].fillna(df_filtered['Promoter'].mode()[0])
df_filtered.loc[:, 'Genre'] = df_filtered['Genre'].fillna(df_filtered['Genre'].mode()[0])

In [817]:
print(df_filtered.dtypes)

Event Date                          object
Headliner                           object
Year                               float64
headliner_monthly_listeners        float64
Support_Total_Monthly_Listeners    float64
monthly_listeners                  float64
Number of Shows                      int64
Support                             object
Venue                               object
City                                object
State                               object
Country                             object
Market                              object
Company Type                        object
Currency                            object
Promoter                            object
Genre                               object
Avg. Tickets Sold                  float64
Avg. Gross USD                     float64
Avg. Event Capacity                float64
Avg. Capacity Sold                  object
Ticket Price Min USD               float64
Ticket Price Max USD               float64
Ticket Pric

In [818]:
# Ensure the Event Date is datetime type
df_filtered['Event Date'] = pd.to_datetime(df_filtered['Event Date'], errors='coerce')

df_filtered['Event Year'] = df_filtered['Event Date'].dt.year
df_filtered['Event Month'] = df_filtered['Event Date'].dt.month
df_filtered['Event Day'] = df_filtered['Event Date'].dt.day
df_filtered['Day of Week'] = df_filtered['Event Date'].dt.dayofweek  # 0=Monday, 6=Sunday

df_filtered.drop(columns=['Event Date'], inplace=True)

In [819]:
# check other feature's missing value
print(df_filtered.isnull().sum())

Headliner                          0
Year                               0
headliner_monthly_listeners        0
Support_Total_Monthly_Listeners    0
monthly_listeners                  0
Number of Shows                    0
Support                            0
Venue                              0
City                               0
State                              0
Country                            0
Market                             0
Company Type                       0
Currency                           0
Promoter                           0
Genre                              0
Avg. Tickets Sold                  0
Avg. Gross USD                     0
Avg. Event Capacity                0
Avg. Capacity Sold                 0
Ticket Price Min USD               0
Ticket Price Max USD               0
Ticket Price Avg. USD              0
Month                              0
day_of_week                        0
Event Year                         0
Event Month                        0
E

# Feature Selection

In [877]:
# Select top 4 numerical features
correlation_matrix = df_filtered.select_dtypes(include=['number']).corr()
top_numeric_features = correlation_matrix['Avg. Gross USD'].abs().sort_values(ascending=False).index[1:6]  # choose top 5
print("Selected Numerical Features:", top_numeric_features)

Selected Numerical Features: Index(['Avg. Tickets Sold', 'Avg. Event Capacity', 'Ticket Price Max USD',
       'Support_Total_Monthly_Listeners', 'monthly_listeners'],
      dtype='object')


In [879]:
# Select all categorical variables
categorical_features = ['Headliner', 'Support', 'Venue', 'Market', 'Promoter', 'Genre']

# Label Encoding
df_encoded = df_filtered.copy()
for col in categorical_features:
    df_encoded[col] = LabelEncoder().fit_transform(df_encoded[col])

# Calculate F value
X_cat = df_encoded[categorical_features]
y = df_encoded['Avg. Gross USD']
selector = SelectKBest(score_func=f_classif, k=5)  # Select the 5 most relevant category features
selector.fit(X_cat, y)

# Get the best category features
best_categorical_features = X_cat.columns[selector.get_support()]
print("Selected Categorical Features:", best_categorical_features)

Selected Categorical Features: Index(['Headliner', 'Support', 'Venue', 'Market', 'Genre'], dtype='object')


In [883]:
# final features
final_features = list(top_numeric_features) + list(best_categorical_features)
print("Final Selected Features:", final_features)

# Create the final data set for training
df_model = df_filtered[final_features + ['Avg. Gross USD']]

Final Selected Features: ['Avg. Tickets Sold', 'Avg. Event Capacity', 'Ticket Price Max USD', 'Support_Total_Monthly_Listeners', 'monthly_listeners', 'Headliner', 'Support', 'Venue', 'Market', 'Genre']


# Feature Engineering & Data Preprocessing

In [886]:
# Standardization
scaler = StandardScaler()
df_model.loc[:, top_numeric_features] = scaler.fit_transform(df_model[top_numeric_features])

In [894]:
df_model = df_filtered.copy()

categorical_features = ['Headliner', 'Support', 'Venue', 'Market', 'Genre']
label_encoders = {}

for col in categorical_features:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])
    label_encoders[col] = le  # save encoder

# choose final numerical & categorical features
final_features = ['Avg. Tickets Sold', 'Avg. Event Capacity', 'Ticket Price Max USD', 
                  'Support_Total_Monthly_Listeners', 'monthly_listeners', 'Headliner', 'Support', 'Venue', 'Market', 'Genre']

# Extract X, y
X = df_model[final_features]
y = df_model['Avg. Gross USD']

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# make sure X_train, X_test just contain num variable
X_train = X_train.select_dtypes(include=np.number)
X_test = X_test.select_dtypes(include=np.number)

In [896]:
print("Train set:", X_train.shape)
print("Test set:", X_test.shape)

Train set: (7217, 10)
Test set: (1805, 10)


In [898]:
print(X_train.dtypes)

Avg. Tickets Sold                  float64
Avg. Event Capacity                float64
Ticket Price Max USD               float64
Support_Total_Monthly_Listeners    float64
monthly_listeners                  float64
Headliner                            int64
Support                              int64
Venue                                int64
Market                               int64
Genre                                int64
dtype: object


## Random Forest

In [901]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# predict
y_pred = model.predict(X_test)

# evaluate
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Mean Absolute Error (MAE): {mae:.5f}")
print(f"R² Score: {r2:.5f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.5f}")

# ---- cross validation ----
cv_mae = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')
cv_r2 = cross_val_score(model, X, y, cv=5, scoring='r2')
cv_rmse = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')

print("\nCross-Validation Results (5-Fold):")
print(f"Mean Absolute Error (MAE): {-np.mean(cv_mae):.5f}")
print(f"R² Score: {np.mean(cv_r2):.5f}")
print(f"Root Mean Squared Error (RMSE): {-np.mean(cv_rmse):.5f}")

Mean Absolute Error (MAE): 32749.16764
R² Score: 0.91774
Root Mean Squared Error (RMSE): 141358.21413

Cross-Validation Results (5-Fold):
Mean Absolute Error (MAE): 34720.28056
R² Score: 0.86076
Root Mean Squared Error (RMSE): 156003.02720


In [750]:
# random forest
df_model = df_filtered.copy()

# Find all categorical variables
categorical_features = df_model.select_dtypes(include=['object']).columns

# Label Encoding
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])
    label_encoders[col] = le  # save encoder

# Make sure X contains only numerical data
X = df_model.drop(columns=['Avg. Gross USD'])
y = df_model['Avg. Gross USD']

# re-split train & test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# just keep numeric variables
X_train = X_train.select_dtypes(include=[np.number])
X_test = X_test.select_dtypes(include=[np.number])

# train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# predict
y_pred = model.predict(X_test)

# evaluate
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Mean Absolute Error (MAE): {mae:.5f}")
print(f"R² Score: {r2:.5f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.5f}")

# ---- Cross-Validation ----
X = X.select_dtypes(include=[np.number])
cv_mae = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')
cv_r2 = cross_val_score(model, X, y, cv=5, scoring='r2')
cv_rmse = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')

print("\nCross-Validation Results (5-Fold):")
print(f"Mean Absolute Error (MAE): {-np.mean(cv_mae):.5f}")
print(f"R² Score: {np.mean(cv_r2):.5f}")
print(f"Root Mean Squared Error (RMSE): {-np.mean(cv_rmse):.5f}")

Mean Absolute Error (MAE): 8800.92919
R² Score: 0.95496
Root Mean Squared Error (RMSE): 104595.01385

Cross-Validation Results (5-Fold):
Mean Absolute Error (MAE): 10721.31390
R² Score: 0.92786
Root Mean Squared Error (RMSE): 111462.92976


In [751]:
print(X.dtypes)

Headliner                            int64
Year                               float64
headliner_monthly_listeners        float64
Support_Total_Monthly_Listeners    float64
monthly_listeners                  float64
Number of Shows                      int64
Support                              int64
Venue                                int64
City                                 int64
State                                int64
Country                              int64
Market                               int64
Company Type                         int64
Currency                             int64
Promoter                             int64
Genre                                int64
Avg. Tickets Sold                  float64
Avg. Event Capacity                float64
Avg. Capacity Sold                   int64
Ticket Price Min USD               float64
Ticket Price Max USD               float64
Ticket Price Avg. USD              float64
Month                                int64
day_of_week